In [ ]:
import statsmodels
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.stattools import acf, pacf
import h5py
import numpy as np
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from multiprocess import Pool

In [ ]:
f = h5py.File("/Users/danielzeiberg/Documents/Human3.6/Processed/all_2D.h5")

In [ ]:
f["instances"].shape

In [ ]:
def do(index = None, subject=None, action=None, subaction=None, camera=None, limit=10000, shouldPlot=True, lastSignificant=True):
    if shouldPlot:
        fig, axes = plt.subplots(32, 1)
        fig.set_size_inches(16, 256)
        plt.subplots_adjust(hspace=.90)
    if index:
        inst = index
    else:
        indices = []
        subjectindices = []
        actionindices = []
        subactionindices = []
        cameraindices = []
        for i in range(f["instances"].shape[0]):
            if subject != None and f["subjects"][i] == subject:
                subjectindices.append(i)
            if action != None and f["actions"][i] == action:
                actionindices.append(i)
            if subaction != None and f["subactions"][i] == subaction:
                subactionindices.append(i)
            if camera != None and f["cameras"][i] == camera:
                cameraindices.append(i)
        if not subject and not action and not subaction and not camera:
            indices = range(f["instances"].shape[0])
        elif subject != None and not len(indices):
            indices = subjectindices
        elif action != None:
            if not len(indices):
                indices = actionindices
            else:
                indices = set(indices).intersection(actionindices)
        elif subaction != None:
            if not len(indices):
                indices = subactionindices
            else:
                indices = set(indices).intersection(subactionindices)
        elif camera != None:
            if not len(indices):
                indices = cameraindices
            else:
                indices = set(indices).intersection(camera)
        inst = np.random.choice(indices)
    dat = f["instances"][inst]
    def plotVals(i,j):
        num = min(f["lengths"][inst], limit)
        corr= dat[:num,i,j]
        if shouldPlot:
            plot_acf(corr,
                        title="Autocorrelation S: {} A: {} SA: {} C: {} sensor({},{})".format(f["subjects"][inst],
                                                                                              f["actions"][inst],
                                                                                              f["subactions"][inst],
                                                                                              f["cameras"][inst],
                                                                                              i,j),
                        lags=range(1,len(corr)),
                        ax = axes[i*2+j],
                        alpha=None
                        )
        acfVals,confInt = acf(corr, nlags=len(corr)-1, alpha=0.05)
#         def sse(lag):
#             return 1.96 / np.sqrt(corr.shape[0] - lag)
#         SE = np.array([sse(lag) for lag in range(len(corr))])
        maxLag = 0
        for idx,lag in enumerate(range(len(corr))):
            if acfVals[idx] < confInt[idx,0]-acfVals[idx] or acfVals[idx] > confInt[idx,1]-acfVals[idx]:
                color="green"
                if lastSignificant:
                    maxLag = idx
            else:
                if not lastSignificant:
                    if maxLag == 0:
                        maxLag = idx - 1
                color="red"
            if shouldPlot:
                axes[2*i+j].scatter(idx, acfVals[idx], color=color, zorder=3)
        if shouldPlot:
            axes[2*i+j].fill_between(np.arange(len(corr)), confInt[:,0]-acfVals, confInt[:,1]-acfVals, alpha=0.25)
            axes[2*i+j].set_xticks(np.arange(0,len(corr), len(corr)//20))
            axes[2*i+j].set_xlabel("lag")
            axes[2*i+j].set_ylabel("Autocorrelation")
            axes[2*i+j].set_xticklabels(np.arange(0,len(corr), len(corr)//20), rotation=45)
        return maxLag
    maxLags = []
    for i in range(16):
        for j in range(2):
            maxLags.append(plotVals(i,j))
    print(np.mean(maxLags))
    return np.mean(maxLags)

In [ ]:
do(lastSignificant=False)

In [ ]:
maxLags = [do(index=i, shouldPlot=False, lastSignificant=False) for i in range(f["instances"].shape[0])]

In [ ]:
plt.hist(maxLags)
plt.title("lag based off first insignificant, mean: {}, median: {} std: {}, min: {}, max: {}".format(
    np.mean(maxLags), np.median(maxLags), np.std(maxLags), np.min(maxLags), np.max(maxLags)))

In [ ]:
def getACF():
    inst = np.random.choice(f["instances"].shape[0])
    dat = f["instances"][inst]
    def getVal(i,j):
        num = f["lengths"][inst]
        corr= dat[:num,i,j]
        acfVals, confint = acf(corr,
            nlags=5,
            qstat = False,
            fft=True,
            alpha=.05,
            
           )
        print(acfVals)
        print(confint)
    for i in range(16):
        for j in range(2):
            getVal(i,j)

In [ ]:
getACF()

In [ ]:
choice = np.random.choice(f["instances"].shape[0])
length = f["lengths"][choice]
dat = f["instances"][choice][:length]
for i in range(16):
    for j in range(2):
        plt.clf()
        plt.plot(dat[0:-1:5,i,j])
        plt.axhline(y=dat[0:-1:5,i,j].mean(),linestyle='-')
        plt.show()

The data looks relatively stationary

In [ ]:
choice = np.random.choice(f["instances"].shape[0])
length = f["lengths"][choice]
dat = f["instances"][choice][:length]
for i in range(16):
    for j in range(2):
        plt.clf()
        plt.plot(dat[0:15,i,j])
        plt.axhline(y=dat[0:15,i,j].mean(),linestyle='-')
        plt.show()

In [ ]:
choice = np.random.choice(f["instances"].shape[0])
(f["instances"][choice, 1:-1:5,:,:] == 0).all() and (f["instances"][choice, 2:-1:5,:,:] == 0).all() and (f["instances"][choice, 3:-1:5,:,:] == 0).all() and (f["instances"][choice, 4:-1:5,:,:] == 0).all()